In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground.DataGenerator import DataGenerator
from bart_playground.params import Tree
from bart_playground.bart import DefaultBART
import bartz

In [3]:
proposal_probs = {"grow" : 0.4,
                  "prune" : 0.4,
                  "change" : 0.1,
                  "swap" : 0.1}
generator = DataGenerator(n_samples=3000, n_features=40, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="linear")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[63.69758676 65.48891713 70.34915697 54.40002918 59.24234782 64.16862716
 69.78577881 68.85612404 53.55980009 60.15631858 61.31876181 66.06233364]


In [4]:
# initialize numba
bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:18<00:00, 16.05it/s]


In [5]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success, acceptance and change rates
df["success_rate"] = df["success"] / df["selected"]
df["accept_rate"] = df["accepted"] / df["success"] # The "acceptance rate" reflects the MH acceptance probability
df["change_rate"] = df["accepted"] / df["selected"] # The "change rate" reflects the frequency of tree changes

print(df)

        selected  success  accepted  success_rate  accept_rate  change_rate
grow       11825    11825      3621      1.000000     0.306216     0.306216
prune      12191    11493      3471      0.942745     0.302010     0.284718
change      3014     2839      1019      0.941938     0.358929     0.338089
swap        2970     1012       377      0.340741     0.372530     0.126936


In [6]:
bart2 = DefaultBART(ndpost=1000, nskip=100, n_trees=50, proposal_probs=proposal_probs)
%prun -s cumtime -D profile_bart.prof -q bart2.fit(X_train, y_train)
!gprof2dot -f pstats profile_bart.prof -o profile_bart.dot
!dot -Tpng profile_bart.dot -o profile_bart.png

Iterations: 100%|██████████| 1100/1100 [00:19<00:00, 57.51it/s]


 
*** Profile stats marshalled to file 'profile_bart.prof'.


In [7]:
arrays = [tree.vars for tree in bart.trace[-1].trees]
counts = np.array([np.count_nonzero(arr >= 0) for arr in arrays])
print(counts)
deep_trees = np.array([count >= 3 for count in counts])
print(np.where(deep_trees))

[2 4 2 2 3 3 3 3 2 4 3 4 3 3 4 1 1 2 2 1 4 2 2 2 2 5 3 2 4 2 2 2 3 5 3 1 4
 2 2 3 2 1 2 1 2 4 3 2 2 1 5 4 2 2 1 3 3 2 1 4 2 1 2 2 2 3 2 4 3 1 2 2 3 1
 1 1 2 2 1 2 2 2 2 3 2 2 2 1 2 2 2 2 3 2 2 3 2 1 4 2]
(array([ 1,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 20, 25, 26, 28, 32, 33,
       34, 36, 39, 45, 46, 50, 51, 55, 56, 59, 65, 67, 68, 72, 83, 92, 95,
       98]),)


In [8]:
np.mean([len(arr) for arr in arrays])

np.float64(8.8)

In [9]:
print(bart.sampler.trace[-1].global_params)

{'eps_sigma2': array([0.00081172])}


In [10]:
from bart_playground import visualize_tree
tree_sp : Tree = bart.sampler.trace[-1].trees[50]

print(tree_sp)
print(tree_sp.vars)
print(tree_sp.leaf_vals)

X_19 <= 0.842776597 (split, n = 2250)
	X_25 <= 0.639114857 (split, n = 1912)
		Val: -0.012581126 (leaf, n = 1239)
		Val: -0.000585763 (leaf, n = 673)
	Val: 0.007525850 (leaf, n = 338)
[19 25 -1 -1 -1 -2 -2 -2]
[        nan         nan  0.00752585 -0.01258113 -0.00058576         nan
         nan         nan]


In [11]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

INFO:2025-06-23 17:59:16,168:jax._src.xla_bridge:867: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Iteration 100/300 P_grow=0.53 P_prune=0.47 A_grow=0.13 A_prune=0.15 (burnin)
Iteration 200/300 P_grow=0.50 P_prune=0.50 A_grow=0.10 A_prune=0.06
Iteration 300/300 P_grow=0.50 P_prune=0.50 A_grow=0.10 A_prune=0.12


In [12]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 2.017674140026814,
 'rf': 17.62572932546826,
 'lr': 0.010385157940898898,
 'btz': 2.043393861149057}

In [13]:
print(bart.sampler.trace[-1].evaluate(X_train)[:12])
print(bart.preprocessor.transform_y(y_train)[:12])

[ 0.09659852  0.09275349  0.18248593 -0.21848202 -0.05988892  0.05594227
  0.20686662  0.19135175 -0.20834809 -0.06192682  0.03009366  0.09309844]
[ 0.05106411  0.09666862  0.22040284 -0.18563737 -0.06235941  0.06305607
  0.2060601   0.18239251 -0.20702831 -0.03909112 -0.00949711  0.11126692]


In [14]:
mean_squared_error(y_test, np.ones_like(y_test) * y_test.mean())

36.67989681136702

In [15]:
if all([(bart.sampler.trace[-1].trees[i].evaluate() == bart.sampler.trace[-1].trees[i].evaluate(X_train)).all()
            for i in range(100)]):
    print("True")
else:
    print("False")

True


In [16]:
if np.allclose(bart.trace[-1].evaluate(), bart.trace[-1].evaluate(X_train), atol=1e-6):
    print("True")
else:
    print("False")

True
